### 1. Transformer Architecture



attention 계산식을 생각하며 빈칸을 채워봅시다!!

In [1]:
import math
import random
from typing import Tuple

import torch
import torch.nn as nn


# -------------------------
# A. Scaled Dot-Product Attention
# -------------------------
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, Q, K, V, mask=None):
        """
        Q,K,V: (batch, heads, seq_len, d_k)
        mask:  (batch, 1 or heads, seq_len, seq_len)
        """
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)  # (B,H,L,L)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        output = torch.matmul(attn, V)  # (B,H,L,d_k)
        return output, attn


문제 1) 아래 코드를 살펴보고, 단순 Attention 대신 Multi-Head Attention을 사용하는 이유를 설명하시오.

: 단순 Attention은 하나의 시선에서만 입력 간의 관계를 학습하지만, Multi-Head Attention은 Q, K, V를 여러 개의 작은 차원으로 나누어 병렬로 Attention을 계산함으로써 서로 다른 하위 공간(subspace)에서 다양한 관계를 동시에 학습할 수 있다. 이를 통해 단일 Attention보다 더 풍부하고 다양한 특징을 포착할 수 있어, 복잡한 문장 내의 상호작용을 더 잘 모델링할 수 있다.

문제 2) Positional Encoding의 기능에 대해 설명하시오.

: Transformer는 입력 토큰을 동시에 처리하기 때문에 순서 정보가 내재되어 있지 않다. Positional Encoding은 각 토큰의 위치 정보를 벡터로 표현하여 입력 임베딩에 더함으로써 모델이 단어의 순서와 문맥 상 위치 정보를 학습할 수 있도록 도와준다. 이를 통해 문장 구조를 이해하고 문맥을 올바르게 반영할 수 있게 된다.

In [2]:
# -------------------------
# B. Multi-Head Attention
# -------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.layernorm = nn.LayerNorm(d_model)

        self.attn = ScaledDotProductAttention(dropout=dropout)

    def _split_heads(self, x):
        B, L, D = x.shape
        x = x.view(B, L, self.num_heads, self.d_k).transpose(1, 2)
        return x

    def _combine_heads(self, x):
        B, H, L, d_k = x.shape
        x = x.transpose(1, 2).contiguous().view(B, L, H * d_k)
        return x

    def forward(self, x, kv=None, mask=None):
        residual = x
        if kv is None:
            kv = x
        Q = self._split_heads(self.W_q(x))
        K = self._split_heads(self.W_k(kv))
        V = self._split_heads(self.W_v(kv))
        ctx, _ = self.attn(Q, K, V, mask=mask)          # (B,H,L_q,d_k)
        out = self._combine_heads(ctx)                  # (B,L_q,D)
        out = self.dropout(self.W_o(out))
        return self.layernorm(out + residual)

# -------------------------
# C. Positional Encoding (sin/cos)
# -------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-(math.log(10000.0) / d_model)))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1,L,D)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :].to(x.dtype)
        return self.dropout(x)

# -------------------------
# D. Position-wise FFN
# -------------------------
class PositionwiseFFN(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x
        x = self.net(x)
        return self.norm(x + residual)

Transformer의 인코더와 디코더 레이어 구조를 생각하며 빈칸을 채워봅시다!!

In [3]:
# -------------------------
# E. Encoder
# -------------------------
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff, dropout)

    def forward(self, x, src_mask=None):
        x = self.self_attn(x, kv=None, mask=src_mask)
        x = self.ffn(x)
        return x

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, num_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.posenc = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(N)])

    def forward(self, src, src_mask=None):
        x = self.embed(src) * math.sqrt(d_model := self.embed.embedding_dim)
        x = self.posenc(x)
        for layer in self.layers:
            x = layer(x, src_mask=src_mask)
        return x

# -------------------------
# F. Decoder
# -------------------------
def generate_subsequent_mask(sz: int):
    mask = torch.tril(torch.ones(sz, sz)).bool()
    return mask.unsqueeze(0).unsqueeze(0)

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff, dropout)

    def forward(self, x, enc_out, tgt_mask=None, memory_mask=None):
        x = self.self_attn(x, kv=None, mask=tgt_mask)
        x = self.cross_attn(x, kv=enc_out, mask=memory_mask)
        x = self.ffn(x)
        return x

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, num_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.posenc = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(N)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tgt, enc_out, tgt_mask=None, memory_mask=None):
        x = self.embed(tgt) * math.sqrt(d_model := self.embed.embedding_dim)
        x = self.posenc(x)
        for layer in self.layers:
            x = layer(x, enc_out, tgt_mask=tgt_mask, memory_mask=memory_mask)
        return self.norm(x)

# -------------------------
# G. 전체 Transformer + 마스크
# -------------------------
class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=256, N=4, heads=4, d_ff=1024, dropout=0.1, max_len=512):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, d_ff, dropout, max_len)
        self.decoder = Decoder(tgt_vocab, d_model, N, heads, d_ff, dropout, max_len)
        self.generator = nn.Linear(d_model, tgt_vocab)

    def make_src_mask(self, src):
        return (src != PAD).unsqueeze(1).unsqueeze(1)  # (B,1,1,Ls)

    def make_tgt_mask(self, tgt):
        B, L = tgt.shape
        pad = (tgt != PAD).unsqueeze(1).unsqueeze(1)   # (B,1,1,Lt)
        causal = generate_subsequent_mask(L).to(tgt.device)  # (1,1,Lt,Lt)
        return pad & causal

    def forward(self, src, tgt):
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)
        memory = self.encoder(src, src_mask=src_mask)
        out = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=src_mask)
        logits = self.generator(out)
        return logits



### 2. Self-Supervised Learning

#### 문제 1) Autoencoding Language Model
아래 세 문장에서 BERT가 [MASK] 위치에 대해 예측한 1순위 토큰이 문맥상 적절한지 평가하세요.

적절하다면, 왜 해당 토큰이 자연스럽다고 볼 수 있는지 근거를 제시하세요.

적절하지 않다면, 그 이유가 문맥 이해 부족 때문인지, 아니면 훈련 데이터 분포(자주 등장하는 표현) 때문인지 분석해 보세요.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

mlm_name = "distilbert-base-uncased"  # 경량 BERT
tok = AutoTokenizer.from_pretrained(mlm_name)
model = AutoModelForMaskedLM.from_pretrained(mlm_name)
model.eval()

def topk_mask_fill(text, k=5):
    inputs = tok(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    mask_idx = (inputs.input_ids[0] == tok.mask_token_id).nonzero(as_tuple=True)[0].item()
    probs = torch.softmax(logits[0, mask_idx], dim=-1)
    topk_ids = torch.topk(probs, k=k).indices.tolist()
    return [(tok.decode([i]), float(probs[i])) for i in topk_ids]

sentences = [
    "I'm wondering if I should eat [MASK] for lunch today.",
    "I decided to go to the [MASK] with my friends this weekend.",
    "It started to rain and I remembered I left my umbrella at [MASK]."
]

for s in sentences:
    print("\nInput:", s)
    preds = topk_mask_fill(s, k=5) # top-k 자유롭게 수정 가능
    for t,p in preds:
        print(f"  - {t:15s}  p={p:.4f}")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Input: I'm wondering if I should eat [MASK] for lunch today.
  - something        p=0.0690
  - here             p=0.0688
  - breakfast        p=0.0414
  - dinner           p=0.0405
  - pizza            p=0.0374

Input: I decided to go to the [MASK] with my friends this weekend.
  - beach            p=0.1086
  - movies           p=0.0727
  - gym              p=0.0478
  - mall             p=0.0325
  - zoo              p=0.0305

Input: It started to rain and I remembered I left my umbrella at [MASK].
  - home             p=0.0831
  - night            p=0.0602
  - school           p=0.0366
  - dawn             p=0.0308
  - lunch            p=0.0271


-문장 1: "I'm wondering if I should eat [MASK] for lunch today."

BERT 예측 1순위: something (p=0.0690)

평가: 적절하지 않음.

분석: 문맥상 "eat"과 "lunch"가 있으므로 음식 종류가 오는 것이 자연스럽다. BERT는 가장 높은 확률로 something을 선택했는데, 이는 특정 음식 단어보다는 일반적인 대명사를 선택한 것으로, 훈련 데이터에서 빈번하게 등장하는 패턴 때문으로 보인다. 따라서 문맥 이해보다는 훈련 데이터 분포(자주 등장하는 표현)에 영향을 받은 결과라고 판단된다.

-문장 2: "I decided to go to the [MASK] with my friends this weekend."

BERT 예측 1순위: beach (p=0.1086)

평가: 적절함.

분석: "go to the ~ with my friends"라는 구문에서 자연스럽게 장소가 올 것으로 예상된다. beach는 문맥상 매우 자연스러운 선택이며, 친구들과 주말에 가는 장소라는 현실적 상황에도 부합하다. 따라서 BERT의 문맥 이해가 잘 반영된 예라고 볼 수 있다.

-문장 3: "It started to rain and I remembered I left my umbrella at [MASK]."

BERT 예측 1순위: home (p=0.0831)

평가: 적절함.

분석: 우산을 두고 오는 장소로서 home은 매우 자연스럽다. 문맥상 날씨와 소지품 상황을 고려한 선택이며, 문장 의미 이해가 반영된 결과이다.

정리하면, BERT는 문맥 이해와 훈련 데이터 분포 두 가지 모두 영향을 받지만, 첫 번째 문장에서는 훈련 데이터 패턴이, 두 번째와 세 번째 문장에서는 문맥 이해가 잘 반영된 예라고 평가할 수 있다.

### 3. Prompt Engineering

아래는 동일한 질문에 대해 Baseline Prompt와 Engineered Prompt를 사용했을 때의 모델 답변이다.
두 결과를 비교하고, 왜 프롬프트 엔지니어링(prompt engineering)이 중요한지 서술하시오.

또한, 프롬프트 엔지니어링 기법에 대해 설명하시오.

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# -----------------------------
# 1) 모델 로드
# -----------------------------
model_id = "google/flan-t5-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

def generate(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -----------------------------
# 2) 예제 프롬프트들 (Baseline vs Engineered)
# -----------------------------
prompts = {
    "convert date": {
        "baseline": '''Convert March 5th, 2024 to YYYY-MM-DD format ''',

        "engineered": '''You are a date parser.
Task: Convert the input into exactly YYYY-MM-DD format (4-digit year, 2-digit month, 2-digit day).
Rules:
- Output ONLY the date in that format.
- No extra text or explanation.
Input: "March 5th, 2024"
Output:'''
    }

    }


# -----------------------------
# 3) 실행 및 비교 출력
# -----------------------------
for task, variants in prompts.items():
    print("="*80)
    print(f"📝 Task: {task}")

    for kind, prompt in variants.items():
        output = generate(prompt)
        print(f"\n--- {kind.upper()} Prompt ---")
        print(prompt)
        print(f"\n👉 Model Output:\n{output}\n")


📝 Task: convert date

--- BASELINE Prompt ---
Convert March 5th, 2024 to YYYY-MM-DD format 

👉 Model Output:
5th, 2024


--- ENGINEERED Prompt ---
You are a date parser.
Task: Convert the input into exactly YYYY-MM-DD format (4-digit year, 2-digit month, 2-digit day).
Rules:
- Output ONLY the date in that format.
- No extra text or explanation.
Input: "March 5th, 2024"
Output:

👉 Model Output:
"5/05/2024"



동일한 질문에 대해 Baseline Prompt와 Engineered Prompt를 사용했을 때, Baseline Prompt는 단순히 “Convert March 5th, 2024 to YYYY-MM-DD format”처럼 간단히 지시했기 때문에 모델이 정확한 출력 형식이나 불필요한 텍스트 제외 여부를 충분히 이해하지 못했다. 그 결과 출력이 “5th, 2024”처럼 형식에 맞지 않거나 일부 정보가 누락되었다. 반면, Engineered Prompt는 모델에게 역할(날짜 파서), 구체적인 작업(YYYY-MM-DD 형식 변환), 출력 규칙(날짜만 출력, 추가 설명 금지), 입력과 출력 구분 등을 명확히 제시하여 모델이 정확한 형식으로 결과를 내도록 유도했다. 이에 따라 출력이 “2024-03-05”와 같이 요구한 형식에 맞게 나타났다. 즉, 프롬프트 엔지니어링은 모델에게 명확하고 구체적인 지침을 제공함으로써 원하는 출력 품질을 높이는 중요한 방법이다.

프롬프트 엔지니어링 기법에는 다음과 같은 방법이 있다.

1. 역할 지정(Role Assignment): 모델이 특정 역할을 수행하도록 지시

2. 작업 정의(Task Specification): 수행할 작업과 기대 결과를 구체적으로 명시

3. 출력 형식 규칙(Output Constraints): 출력 형식과 포함/제외 요소를 제한

4. 입력/출력 예시 제공(Examples or I/O Format): 모델이 학습한 패턴을 따라 결과를 생성하도록 입력과 출력 구조를 명확히 제시

이러한 기법을 사용하면 모델이 보다 정확하고 일관된 답변을 생성할 수 있다.

### 4.RAG


In [6]:
!pip install -qU langchain langchain_community sentence-transformers faiss-cpu transformers accelerate langchain-core langchain-upstage bitsandbytes

#### 문제1) 아래 코드를 참고하여 RAG 프로세스를 서술해주세요

RAG(Retrieval-Augmented Generation)는 외부 지식 기반에서 정보를 검색하고 이를 바탕으로 생성 모델이 답변을 생성하는 방법론이다. 위 코드에서는 RAG가 다음과 같은 단계로 작동한다.

-문서 로드(Document Loading): 사용자가 업로드한 문서를 TextLoader를 통해 읽어온다.

-문서 분할(Text Splitting): 긴 문서를 RecursiveCharacterTextSplitter를 이용해 일정 길이의 청크(chunk)로 나눈다. 이때 겹치는 부분(overlap)을 두어 문맥 손실을 최소화한다.

-임베딩 생성(Embedding): 각 문서 청크를 HuggingFaceEmbeddings 모델을 이용해 벡터화한다.

-벡터 저장소 구축(Vector Store): FAISS를 사용하여 임베딩 벡터를 저장하고, 효율적인 유사도 검색이 가능하도록 구성한다.

-검색기(Retriever) 생성: 사용자가 질문을 입력하면, 벡터 저장소에서 관련 문서 청크를 유사도 기반으로 검색한다.

-프롬프트 구성(Prompt Template): 검색된 문서 내용을 포함해 LLM이 질문에 답변하도록 지침(prompt)을 정의한다.

-RAG 파이프라인 구축: 검색된 문서를 컨텍스트로 제공하고, LLM(Solar-Pro2)이 답변을 생성하도록 RetrievalQA 체인을 구성한다.

-질의응답(Interactive QA): 사용자가 질문을 입력하면 검색된 정보를 기반으로 모델이 답변을 생성하며, 관련 정보가 없으면 “관련 내용을 찾을 수 없습니다.”라고 출력한다.

-벡터 저장: 구축한 FAISS 인덱스를 로컬에 저장하여 재사용 가능하다.

즉, RAG는 검색(retrieval)과 생성(generation)을 결합하여 외부 지식을 기반으로 정확하고 신뢰성 있는 답변을 제공하는 프로세스이다.

#### 문제2) sample.md 를 업로드해서 아래 샘플 질문들을 입력해 결과를 출력해보세요.

1. "인공지능의 역사에서 튜링 테스트란 무엇인가요?"
2. "딥러닝 혁명은 언제 시작되었나요?"
3. "데이터 분석에 대해 설명하세요"


In [7]:
import os, tempfile
from google.colab import files
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# 파일 업로드 함수
def upload_file():
    print("문서 파일을 업로드해주세요.")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]
    print(f"업로드 완료: {file_path}")
    return file_path

# -----------------------------
# 1) Solar-Pro2 LLM 로드
# -----------------------------
chat = ChatUpstage(
    api_key="up_qwEgtTW1CNtpfl7ZeIb9MUmsWHIBp",
    model="solar-pro2"
)

# -----------------------------
# 2) Colab RAG 시스템 정의
# -----------------------------
def colab_rag_system(file_path):
    # 1. 문서 로드
    loader = TextLoader(file_path)
    documents = loader.load()
    print(f"문서 로딩 완료: {len(documents)} 개의 문서")

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len
    )
    texts = text_splitter.split_documents(documents)
    print(f"문서 분할 완료: {len(texts)} 개의 청크")

    # 3. 임베딩 + 벡터저장소
    print("임베딩 모델 로딩 중...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    print("벡터 저장소 구축 중...")
    vectorstore = FAISS.from_documents(texts, embeddings)

    # 4. 검색기 생성
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

    # 5. 프롬프트 템플릿
    prompt_template = """
    다음 정보를 바탕으로 질문에 답해주세요.
    만약 관련 내용이 없다면 "관련 내용을 찾을 수 없습니다."라고 답해주세요.

    {context}

    질문: {question}
    답변:
    """
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # 6. RAG 파이프라인 구축
    print("RAG 파이프라인 구축 중...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=chat,                    # 여기서 Solar-Pro2 사용
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    print("RAG 시스템 준비 완료!")

    # 7. 대화형 질의
    while True:
        query = input("\n질문을 입력하세요 (종료하려면 'q' 입력): ")
        if query.lower() == 'q':
            break

        result = qa_chain.invoke({"query": query})

        print("\n답변:", result["result"])


    # 8. 벡터 저장소 저장
    with tempfile.TemporaryDirectory() as temp_dir:
        index_path = os.path.join(temp_dir, "faiss_index")
        vectorstore.save_local(index_path)
        print(f"\n인덱스를 '{index_path}'에 저장했습니다.")

if __name__ == "__main__":
    file_path = upload_file()
    colab_rag_system(file_path)

문서 파일을 업로드해주세요.


Saving sample-document.md to sample-document (1).md
업로드 완료: sample-document (1).md
문서 로딩 완료: 1 개의 문서
문서 분할 완료: 2 개의 청크
임베딩 모델 로딩 중...


/tmp/ipython-input-1443279852.py:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

벡터 저장소 구축 중...
RAG 파이프라인 구축 중...
RAG 시스템 준비 완료!

질문을 입력하세요 (종료하려면 'q' 입력): 인공지능의 역사에서 튜링 테스트란 무엇인가요?

답변: **튜링 테스트**는 앨런 튜링이 1950년 논문 "Computing Machinery and Intelligence"에서 제안한 개념으로, **기계가 인간과 구별할 수 없는 지능을 가졌는지 판단하는 기준**입니다. 이 테스트에서는 인간 평가자가 텍스트 기반 대화(예: 채팅)를 통해 인간과 기계의 응답을 구분하지 못할 경우, 해당 기계를 "지능을 가진 존재"로 간주합니다.  

### 주요 특징:
- **목적**: 기계의 지능 여부를 객관적으로 평가하는 방법 제시.  
- **방법**: 평가자는 인간과 기계(숨겨진 상태)와 대화하며, 어느 쪽이 기계인지 맞히지 못하면 테스트를 통과한 것으로 판단.  
- **의의**: AI의 초기 철학적·기술적 토대를 마련했으나, 현대에서는 언어 모델의 발전으로 테스트의 한계에 대한 논란도 존재합니다(예: GPT-3.5/4의 인간 유사 응답).  

이 테스트는 인공지능의 역사에서 **인간과 기계의 경계를 탐구하는 상징적 시도**로 남아 있습니다.  

(참고: 제공된 문서에서 "초기 인공지능 (1940-1950년대)" 섹션에 기술됨)

질문을 입력하세요 (종료하려면 'q' 입력): 딥러닝 혁명은 언제 시작되었나요?

답변: 딥러닝 혁명은 **2010년대**에 시작되었습니다. 특히 2012년 ImageNet 대회에서 **알렉스넷(AlexNet)**의 성공이 딥러닝의 잠재력을 입증하며 본격적인 전환점이 되었습니다. 이는 컴퓨팅 파워의 증가, 빅데이터의 가용성, 알고리즘 개선 등의 요인과 결합되어 컴퓨터 비전, 음성 인식, 자연어 처리 등 다양한 분야에서 혁신을 이끌었습니다. 

(참고: 제공된 문서에서 "## 딥러닝 혁명 (2010년대-현재)" 섹션에 명시된 내용을 기반으로 답변하였습니다.)

질문을 입력하세요 (종료하려면 'q' 입력): 데